# Welcome to the Lab 🥼🧪

## 1000+ Portfolio Acquisitions/Dispositions Analysis

We will be examining 1000+ portfolio acquisitions, dispositions, and resale inventory. Let's begin. 

**Note** This notebook will work with any of the 70k+ markets supported by the Parcl Labs API.

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/inspiration/investors/port_1000_plus_acq_disp.ipynb)

In [1]:
# Environment setup
import os
import sys
import subprocess
from datetime import datetime

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido numpy
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')

In [35]:
import parcllabs
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from parcllabs import ParclLabsClient

print(f"Parcl Labs Version: {parcllabs.__version__}")

Parcl Labs Version: 0.4.0


In [3]:
# Add custom charting
cur_dir = os.getcwd()
chart_dir = os.path.join(cur_dir, '../..')

sys.path.append(chart_dir)

from charting.utils import create_labs_logo_dict, format_metro_names

In [4]:
# init client
client = ParclLabsClient(api_key=api_key)

In [5]:
# Get top 100 CBSAs by population
markets = client.search_markets.retrieve(
    location_type='CBSA',
    as_dataframe=True,
    sort_by='TOTAL_POPULATION',
    sort_order='DESC',
    params={
        'limit': 100
    }
)

markets.head()

,parcl_id,country,geoid,state_fips_code,name,state_abbreviation,region,location_type,total_population,median_income,parcl_exchange_market,pricefeed_market,case_shiller_10_market,case_shiller_20_market
0,2900187,USA,35620,None,"New York-Newark-Jersey City, Ny-Nj-Pa",None,None,CBSA,19908595,93610,0,1,1,1
1,2900078,USA,31080,None,"Los Angeles-Long Beach-Anaheim, Ca",None,None,CBSA,13111917,89105,0,1,1,1
2,2899845,USA,16980,None,"Chicago-Naperville-Elgin, Il-In-Wi",None,None,CBSA,9566955,85087,0,1,1,1
3,2899734,USA,19100,None,"Dallas-Fort Worth-Arlington, Tx",None,None,CBSA,7673379,83398,0,1,0,1
4,2899967,USA,26420,None,"Houston-The Woodlands-Sugar Land, Tx",None,None,CBSA,7142603,78061,0,1,0,0


In [12]:
# get housing events for top 100 CBSA's for 
# portfolios of 1000 or more units
# get the most recent data from April

data = client.portfolio_metrics_sf_housing_event_counts.retrieve_many(
    parcl_ids=markets['parcl_id'].tolist(),
    portfolio_size='PORTFOLIO_1000_PLUS',
    as_dataframe=True,
    params={'limit': 1}
)

|████████████████████████████████████████| 100/100 [100%] in 14.1s (7.07/s) 


In [18]:
# join name
df = data.merge(markets[['name', 'parcl_id']], on='parcl_id')
df['name'] = df['name'].apply(lambda x: format_metro_names(x, include_state=True))
# save to csv if you want, uncomment the line below
# df.to_csv('institutional_housing_events.csv', index=False)

In [54]:
# load assets
labs_logo_dict = create_labs_logo_dict(
    src='labs',
    y=1.015
)

In [55]:
charting_data = df[['name', 'acquisitions', 'dispositions', 'new_listings_for_sale', 'new_rental_listings']]
# format col names
charting_data = charting_data.rename(columns={
    'name': 'Metro',
    'acquisitions': 'Acquisitions',
    'dispositions': 'Dispositions',
    'new_listings_for_sale': 'New Listings for Sale',
    'new_rental_listings': 'New Rental Listings'
})

# sort on acquisitions to show the most acquiriing markets first
charting_data = charting_data.sort_values(by='Acquisitions', ascending=False)

In [60]:
# define the table
# Function to normalize column values for color scale
def normalize_column(column):
    col_min = np.min(column)
    col_max = np.max(column)
    return [(val - col_min) / (col_max - col_min) for val in column]

# Normalizing data for each column
normalized_acquisitions = normalize_column(charting_data['Acquisitions'])
normalized_dispositions = normalize_column(charting_data['Dispositions'])
normalized_new_listings_for_sale = normalize_column(charting_data['New Listings for Sale'])
normalized_new_rental_listings = normalize_column(charting_data['New Rental Listings'])

# Define the table with new column headers and improved readability
fig = go.Figure(
    data=[go.Table(
        header=dict(
            values=['<b>Metro</b>', '<b>Acquisitions</b>', '<b>Dispositions</b>', '<b>New Listings for Sale</b>', '<b>New Rental Listings</b>'],
            fill_color='#000000',
            font=dict(color='#FFFFFF', size=14, family="Arial, sans-serif"),
            align='center',
            height=35
        ),
        cells=dict(
            values=[
                [f"{name}" for name in charting_data['Metro'].values],
                charting_data['Acquisitions'],
                charting_data['Dispositions'],
                charting_data['New Listings for Sale'],
                charting_data['New Rental Listings']
            ],
            fill=dict(
                color=[
                    ['#000000']*len(charting_data.index),
                    [f"rgba(46, 64, 87, {v})" for v in normalized_acquisitions],
                    [f"rgba(59, 81, 106, {v})" for v in normalized_dispositions],
                    [f"rgba(72, 99, 125, {v})" for v in normalized_new_listings_for_sale],
                    [f"rgba(84, 117, 144, {v})" for v in normalized_new_rental_listings]
                ]
            ),
            font=dict(
                color='#FFFFFF',
                size=12,  # Slightly increased size for better readability
                family="Arial, sans-serif"
            ),
            align='center',
            height=30
        ))
])

# Adding logo image
fig.add_layout_image(labs_logo_dict)

# Updating the layout with the new title and adjusted dimensions
fig.update_layout(
    title={
        'text': 'Institutional Activity Review: April, 2024',
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=20, color='#FFFFFF', family="Arial, sans-serif")
    },
    width=1000,  # Increased width for better fit of new headers
    height=1800,  # Adjusted height to accommodate 100 markets
    paper_bgcolor='#000000',
    margin=dict(l=10, r=10, t=120, b=10)
)

# Display the figure
fig.show()

# Save the plot
fig.write_image('../graphics/april_institutional_activity.png', width=1000, height=1800)